# Getting to know your data with Pandas

## Pandas

Pandas is the Python Data Analysis Library. 

Pandas is an extremely versatile tool for manipulating datasets.   

It also produces high quality plots with matplotlib, and integrates nicely with other libraries that expect NumPy arrays.

The most important tool provided by Pandas is the **data frame.**

A data frame is a table in which each row and column is given a label.

Pandas DataFrames are documented at:

http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.html

## Getting started

In [3]:
import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from datetime import datetime

#pd.__version__

%matplotlib inline

ImportError: No module named 'pandas_datareader'

## Fetching, storing and retrieving your data

For demonstration purposes, we'll use a library built-in to Pandas that fetches data from standard online sources, such as Yahoo! Finance.

More information on what types of data you can fetch is at:
http://pandas.pydata.org/pandas-docs/stable/remote_data.html

In [ ]:
stocks = 'YELP'
data_source = 'yahoo'
start = datetime(2015,1,1)
end = datetime(2015,12,31)

yahoo_stocks = web.DataReader(stocks, data_source, start, end)

# yahoo_stocks.head()
yahoo_stocks.info()

In [ ]:
yahoo_stocks

### Reading data from a .csv file

In [ ]:
yahoo_stocks.to_csv('yahoo_data.csv')
# print(open('yahoo_data.csv').read())

In [ ]:
df = pd.read_csv('yahoo_data.csv')
df

The number of rows in the DataFrame:

In [ ]:
len(df)

## Working with data columns

The columns or "features" in your data

In [ ]:
df.columns

Selecting a single column from your data

In [ ]:
df['Open']

Another way of selecting a single column from your data

In [ ]:
df.Open

In [ ]:
df[['Open','Close']].head()

In [ ]:
df.Date.head(10)

In [ ]:
df.Date.tail(10)

Changing the column names:

In [ ]:
new_column_names = [x.lower().replace(' ','_') for x in df.columns]
df.columns = new_column_names
df.info()

Now **all** columns can be accessed using the **dot** notation:

In [ ]:
df.adj_close.head()

## Data Frame methods

A DataFrame object has many useful methods.

In [ ]:
df.mean()

In [ ]:
df.std()

In [ ]:
df.median()

In [ ]:
df.open.mean()

In [ ]:
df.high.mean()

### Plotting methods

In [ ]:
df.adj_close.plot(label='adj')
df.low.plot(label='close')
plt.legend(loc='best')

In [ ]:
df.adj_close.hist()

### Bulk Operations

Methods like **sum( )** and **std( )** work on entire columns. 

We can run our own functions across all values in a column (or row) using **apply( )**.

In [ ]:
df.info()

In [ ]:
df.date.head()

The **values** property of the column returns a list of values for the column. Inspecting the first value reveals that these are strings with a particular format.

In [ ]:
first_date = df.date.values[0]
first_date

In [ ]:
datetime.strptime(first_date, "%Y-%m-%d")

In [ ]:
df.date = df.date.apply(lambda d: datetime.strptime(d, "%Y-%m-%d"))
df.date.head()

Each row in a DataFrame is associated with an index, which is a label that uniquely identifies a row.

The row indices so far have been auto-generated by pandas, and are simply integers starting from 0. 

From now on we will use dates instead of integers for indices -- the benefits of this will show later. 

Overwriting the index is as easy as assigning to the **`index`** property of the DataFrame.

In [ ]:
df.index = df.date
df.head()

Now that we have made an index based on date, we can drop the original `date` column.

In [ ]:
df = df.drop(['date'],axis=1)
df.info()

### Accessing rows of the DataFrame

So far we've seen how to access a column of the DataFrame.  To access a row we use a different notation.

To access a row by its index value, use the **`.ix()`** method.

In [ ]:
df.ix[datetime(2015,1,23,0,0)]

To access a row by its sequence number (ie, like an array index), use **`.iloc()`** ('Integer Location')

In [ ]:
df.iloc[0,:]

To iterate over the rows, use **`.iterrows()`**

In [ ]:
num_positive_days = 0
for idx, row in df.iterrows():
    if row.close > row.open:
        num_positive_days += 1
        
print("The total number of positive-gain days is {}.".format(num_positive_days))

## Filtering

It is very easy to select interesting rows from the data.  

All these operations below return a new DataFrame, which itself can be treated the same way as all DataFrames we have seen so far.

In [ ]:
tmp_high = df.high > 55
tmp_high.head()

Summing a Boolean array is the same as counting the number of **`True`** values.

In [ ]:
sum(tmp_high)

Now, let's select only the rows of **`df1`** that correspond to **`tmp_high`**

In [ ]:
df[tmp_high]

Putting it all together, we have the following commonly-used patterns:

In [ ]:
positive_days = df[df.close > df.open]
positive_days.head()

In [ ]:
very_positive_days = df[df.close-df.open > 4]
very_positive_days.head()

## Creating new columns

To create a new column, simply assign values to it.  Think of the columns as a dictionary:

In [ ]:
df['profit'] = (df.open < df.close)
df.head()

In [ ]:
for idx, row in df.iterrows():
    if row.close > row.open:
        df.ix[idx,'gain']='negative'
    elif (row.open - row.close) < 1:
        df.ix[idx,'gain']='small_gain'
    elif (row.open - row.close) < 6:
        df.ix[idx,'gain']='medium_gain'
    else:
        df.ix[idx,'gain']='large_gain'
df.head()

Here is another, more "functional", way to accomplish the same thing.

Define a function that classifies rows, and **`apply`** it to each row.

In [ ]:
def namerow(row):
    if row.close > row.open:
        return 'negative'
    elif (row.open - row.close) < 1:
        return 'small_gain'
    elif (row.open - row.close) < 6:
        return 'medium_gain'
    else:
        return 'large_gain'

df['test_column'] = df.apply(namerow, axis = 1)


In [ ]:
df.head()

OK, point made, let's get rid of that extraneous `test_column`:

In [ ]:
df.drop('test_column', axis = 1)

## Grouping

An **extremely** powerful DataFrame method is **`groupby()`**. 

This is entirely analagous to **`GROUP BY`** in SQL.

It will group the rows of a DataFrame by the values in one (or more) columns, and let you iterate through each group.

Here we will look at the average gain among the  categories of gains (negative, small, medium and large) we defined above and stored in column `gain`.

In [ ]:
gain_groups = df.groupby('gain')

Essentially, **`gain_groups`** behaves like a dictionary
* whose keys are the unique values found in the `gain` column, and 
* whose values are DataFrames that contain only the rows having the corresponding unique values.

In [ ]:
for gain, gain_data in gain_groups:
    print(gain)
    print(gain_data.head())
    print('=============================')

In [ ]:
for gain, gain_data in df.groupby("gain"):
    print('The average closing value for the {} group is {}'.format(gain,
                                                           gain_data.close.mean()))

## Other Pandas Classes

A DataFrame is essentially an annotated 2-D array.

Pandas also has annotated versions of 1-D and 3-D arrays.

A 1-D array in Pandas is called a **`Series`**.

A 3-D array in Pandas is called a **`Panel`**.

To use these, read the documentation!

## Comparing multiple stocks

As a last task, we will use the experience we obtained so far -- and learn some new things -- in order to compare the performance of different stocks we obtained from Yahoo finance.

In [ ]:
stocks = ['ORCL', 'TSLA', 'IBM','YELP', 'MSFT']
attr = 'Close'
df = web.DataReader(stocks, 
                    data_source,                               
                    start=datetime(2014, 1, 1), 
                    end=datetime(2014, 12, 31))[attr]
df.head()

In [ ]:
df.ORCL.plot(label = 'oracle')
df.TSLA.plot(label = 'tesla')
df.IBM.plot(label = 'ibm')
df.MSFT.plot(label = 'msft')
df.YELP.plot(label = 'yelp')
plt.legend(loc='best')

Next, we will calculate returns over a period of length $T$, defined as:

$$r(t) = \frac{f(t)-f(t-T)}{f(t)} $$

The returns can be computed with a simple DataFrame method **`pct_change()`**.  Note that for the first $T$ timesteps, this value is not defined (of course):

In [ ]:
rets = df.pct_change(30)
rets.iloc[25:35]

Now we'll plot the timeseries of the returns of the different stocks.

Notice that the `NaN` values are gracefully dropped by the plotting function.

In [ ]:
rets.ORCL.plot(label = 'oracle')
rets.TSLA.plot(label = 'tesla')
rets.IBM.plot(label = 'ibm')
rets.MSFT.plot(label = 'msft')
rets.YELP.plot(label = 'yelp')
plt.legend(loc='best')

In [ ]:
plt.scatter(rets.TSLA, rets.YELP)
plt.xlabel('TESLA 30-day returns')
plt.ylabel('YELP 30-day returns')

There appears to be some (fairly strong) correlation between the movement of TSLA and YELP stocks.  Let's measure this.

The correlation coefficient between variables $X$ and $Y$ is defined as follows:

$$\text{Corr}(X,Y) = \frac{E\left[(X-\mu_X)(Y-\mu_Y)\right]}{\sigma_X\sigma_Y}$$

Pandas provides a DataFrame method to compute the correlation coefficient of all pairs of columns: **`corr()`**.

In [ ]:
rets.corr()

It takes a bit of time to examine that table and draw conclusions.  

To speed that process up it helps to visualize the table.

We will learn more about visualization later, but for now this is a simple example.

In [ ]:
sns.heatmap(rets.corr(), annot=True)

Finally, it is important to know that the plotting performed by Pandas is just a layer on top of `matplotlib` (i.e., the `plt` package).  

So Panda's plots can (and should) be replaced or improved by using additional functions from `matplotlib`.

For example, suppose we want to know both the returns as well as the standard deviation of the returns of a stock (i.e., its risk).  

Here is visualization of the result of such an analysis, and we construct the plot using only functions from `matplotlib`.

In [ ]:
plt.scatter(rets.mean(), rets.std())
# plt.xlabel('Expected returns')
# plt.ylabel('Standard Deviation (Risk)')
# for label, x, y in zip(rets.columns, rets.mean(), rets.std()):
#    plt.annotate(
#        label, 
#        xy = (x, y), xytext = (20, -20),
#        textcoords = 'offset points', ha = 'right', va = 'bottom',
#        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
#        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

To understand what these functions are doing, (especially the `annotate` function), you will need to consult the online documentation for matplotlib.  Just use Google to find it.